# Tutorial 5 -- Human Detection 
This tutorial will show you how to detect human with the robot. We will use multi-thread to capture the color images and depth images. Then the ssd mobilenet v2 nerual network structure is used for human detection. To enable real-time performance, the tensorRT deep learning framework is used. More information can be found in the nvidia's jetbot project website: https://github.com/NVIDIA-AI-IOT/jetbot

### Step 1 load the pre-trained object detection model

Before running the program, we need to download the pre-trained detection model and put it in the current folder. The model can be found in the following link:https://drive.google.com/file/d/1KjlDMRD8uhgQmQK-nC2CZGHFTbq4qQQH/view

In [1]:
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import cv2
import os
import ctypes
    
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])

def bgr8_to_ssd_input(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1)).astype(np.float32)
    x -= mean[:, None, None]
    x /= stdev[:, None, None]
    return x[None, ...]

class ObjectDetector(object):
    
    def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
        logger = trt.Logger()
        trt.init_libnvinfer_plugins(logger, '')
        load_plugins()
        self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
        self.preprocess_fn = preprocess_fn
        
    def execute(self, *inputs):
        trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
        return parse_boxes(trt_outputs)
    
    def __call__(self, *inputs):
        return self.execute(*inputs)

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')


### Step 2 Initialize the camera instance for the Intel realsense sensor D435i

In [2]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of these two values will be captured by traitlets
    color_value = traitlets.Any()
    depth_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        self.stop_flag = False
        
        #start capture the first depth and color image
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        depth_frame = frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap    

    def _capture_frames(self):
        while(self.stop_flag==False):
            frames = self.pipeline.wait_for_frames()
            color_frame = frames.get_color_frame()
            image = np.asanyarray(color_frame.get_data())
            self.color_value = image

            depth_frame = frames.get_depth_frame()              
            depth_image = np.asanyarray(depth_frame.get_data())
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap
    
    def start(self):
        if not hasattr(self, 'thread') or not self.thread.isAlive():
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        self.stop_flag = True
        if hasattr(self, 'thread'):
            self.thread.join()        
        if self.pipeline_started:
            self.pipeline.stop()

def bgr8_to_jpeg(value,quality=75):
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data

### Step 3 Perform object detection and display the results on widgets
Human is labeled is 1 in the pretrained model. A full list of the detection class indices can be found in the following link https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt

The following program will ask the robot to move forward if a human is detected in the captured color image.

In [18]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML

width = 640
height = 480

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
display(widgets.VBox([
    widgets.HBox([image_widget,]),
    label_widget
]))

import time
from RobotClass import Robot

DEPTH_THRESHOLD = 100
# TODO: EXPERIMENT WITH THRESHOLD TO SEE HOW IT AFFECTS TURNING
SIDE_DIFF_TURN_THRESHOLD = 0.1
FOLLOW_SPEED = 0.3
FOLLOW_TURN_SPEED = 0.3

#initialize the Robot class
robot = Robot()
def processing(change):
    image = change['new']
    depth_value = camera.depth_value
     
    imgsized= cv2.resize(image,(300,300))
    # compute all detected objects
    detections = model(imgsized)
    
    detected_humans = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    if (len(detected_humans) == 0):
        robot.stop()
#         print("NO HUMAN IN VISION")
#         TODO: SELECT THE CLOSEST HUMAN INSTEAD
#     for human in detected_humans:
    else:
        human = detected_humans[0]
        bbox = human['bbox']
#         RESCALE BOX
        left = bbox[0]
        top = bbox[1]
        right = bbox[2]
        bottom = bbox[3]
        scaled_left = int(width * left)
        scaled_top = int(height * top)
        scaled_right = int(width * right)
        scaled_bottom = int(height * bottom)
        scaled_bbox=  [(scaled_left, scaled_top), (scaled_right, scaled_bottom)]
        scaled_height = scaled_bottom - scaled_top
        scaled_width = scaled_right - scaled_left
#         print(scaled_left,scaled_top,scaled_right,scaled_bottom)
#         DRAW BOX ON DEPTH IMAGE
        cv2.rectangle(depth_value,*scaled_bbox , (255, 0, 0), 2)
#         cv2.rectangle(image,*scaled_bbox , (255, 0, 0), 2)
        
#         CHECKING IF DISTANCE FROM LEFT IS GREATER THAN THE RIGHT
# TODO: STOP REDUNANT CALLS TO LEFT/FORWARD/RIGHT
        side_diff = left - 1 + right
#         print(side_diff)
        if (side_diff > SIDE_DIFF_TURN_THRESHOLD):
            robot.right(FOLLOW_TURN_SPEED)
            print("TURNING RIGHT")
        elif (side_diff < -SIDE_DIFF_TURN_THRESHOLD):
            robot.left(FOLLOW_TURN_SPEED)
            print("TURNING LEFT")
        else:
            human_in_range = False
#             for i in range(scaled_height):
#                 for j in range(scaled_width):
# #                     print("BOX_COORDS", i,j)
#                     y = scaled_top + i
#                     x = scaled_left + j 
# #                     print(depth_value[y][x][0])
#                     if depth_value[y][x][0] < DEPTH_THRESHOLD:
#                         human_in_range= True
#                         break
#                 if human_in_range:
#                     break

#             human_in_range =  depth_value[scaled_top+int(scaled_height/2)][scaled_left+int(scaled_width/2)][0] < DEPTH_THRESHOLD:
            if human_in_range:
                robot.stop()
                print("STOPPED AT SAFE DISTANCE")
            else:
                robot.forward(FOLLOW_SPEED)
                print("FOLLOWING")
        

#     image_widget.value = bgr8_to_jpeg(image)
    image_widget.value = bgr8_to_jpeg(depth_value)
    
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
processing({'new': camera.color_value})
camera.observe(processing, names='color_value')

TURNING LEFT
TURNING LEFT
TURNING LEFT
FOLLOWING
TURNING LEFT
TURNING LEFT
TURNING LEFT
FOLLOWING
FOLLOWING
FOLLOWING
FOLLOWING
FOLLOWING
FOLLOWING
FOLLOWING
FOLLOWING
FOLLOWING
FOLLOWING
FOLLOWING
FOLLOWING
TURNING LEFT
TURNING LEFT
FOLLOWING
TURNING LEFT
TURNING LEFT
TURNING RIGHT
TURNING LEFT
TURNING LEFT
TURNING LEFT
TURNING LEFT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNING RIGHT
TURNIN

The following code can be used to stop the capturing of the image and the moving of the robot

In [17]:
camera.unobserve_all()
time.sleep(1.0)
robot.stop()

## Tasks

### 1. Please try to calculate the distance between the detected human and the robot using the depth image. (Note: You can refer to tutorial 2 to get the depth information for a specific point)
### 2. Please try to add collision avoidance function into this program to protect the robot. (Note: You can refer to tutorial 4 for the collision avoidance)
### 3. Think about how to control the robot to move towards the detected human